In [10]:
import os
import sys

sys.path.append(os.path.abspath("../src"))

In [ ]:
import torch


def bool_matmul(a, b):
    a = a.to(torch.uint8)
    b = b.to(torch.uint8)
    c = a @ b
    return c.to(torch.bool)

Sign of derivative can be derived from the weight value, so boolean results are enough.

In [ ]:
def nor_and_eval(weights, inputs):
    weights = weights.to(torch.int32)
    inputs = inputs.to(torch.int32)
    or_and = (weights @ inputs).to(torch.bool)
    return ~or_and


def nor_and_grad(weights, inputs):
    weights = weights.to(torch.int32)
    inputs = inputs.transpose(-1, -2)
    prods = weights * inputs.to(torch.int32)
    residues = prods.sum(dim=1, keepdim=True) - prods
    return (~residues.to(torch.bool) & inputs).to(torch.bool)

In [ ]:
_weights = torch.tensor([[False, True], [True, False]], dtype=torch.bool)
inputs = torch.tensor([[True], [True]], dtype=torch.bool)
nor_and_eval(_weights, inputs)
nor_and_grad(_weights, inputs)

tensor([[False,  True],
        [ True, False]])

In [ ]:
class NorAndLinear:
    def __init__(self, weights):
        self._weights = weights

    def eval(self, inputs):
        weights = self._weights.to(torch.int32)
        inputs = inputs.to(torch.int32)
        or_and = (weights @ inputs).to(torch.bool)
        return ~or_and

    def grad(self, inputs):
        weights = self._weights.to(torch.int32)
        inputs = inputs.transpose(-1, -2)
        prods = weights * inputs.to(torch.int32)
        residues = prods.sum(dim=1, keepdim=True) - prods
        return (~residues.to(torch.bool) & inputs).to(torch.bool)